In [4]:
pip install numpy pandas wget bs4

Note: you may need to restart the kernel to use updated packages.


In [6]:
import wget
wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv')

100% [....................................................................................] 45 / 45

'exchange_rate.csv'

In [9]:
# Import Libraries

In [77]:
import requests
import sqlite3
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

# Suppress generated warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

In [78]:
#Task 1 Logging Functions

In [79]:
def log_progress(message):

    timestamp_format = "%Y-%h-%d-%H:%M:%S" # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open(log_file,"a") as f:
        f.write(timestamp + " : " + message + "\n")

In [80]:
# Extraction of Data

In [81]:
def extract(url, table_attribs):
   
    page = requests.get(url).text
    soup = BeautifulSoup(page, "html.parser")

    df = pd.DataFrame(columns=table_attribs)

    tables = soup.find_all("tbody")
    rows = tables[0].find_all("tr")

    for row in rows:
        col = row.find_all("td")
        if len(col) != 0:
            data_dict = {"Name": col[1].find_all("a")[1]["title"],
                         "MC_USD_Billion": float(col[2].contents[0][:-1])}
            df1 = pd.DataFrame(data_dict, index=[0])
            df = pd.concat([df, df1], ignore_index=True)

    return df

In [82]:
#Task 3 Transforming Data

In [83]:
def transform(df, csv_path):

    # Read exchange rate CSV file
    exchange_rate = pd.read_csv(csv_path)

    # Convert to a dictionary with "Currency" as keys and "Rate" as values
    exchange_rate = exchange_rate.set_index("Currency").to_dict()["Rate"]

    # Add MC_GBP_Billion, MC_EUR_Billion, and MC_INR_Billion
    # columns to dataframe. Round off to two decimals
    df["MC_GBP_Billion"] = [np.round(x * exchange_rate["GBP"], 2) for x in df["MC_USD_Billion"]]
    df["MC_EUR_Billion"] = [np.round(x * exchange_rate["EUR"], 2) for x in df["MC_USD_Billion"]]
    df["MC_INR_Billion"] = [np.round(x * exchange_rate["INR"], 2) for x in df["MC_USD_Billion"]]

    return df

In [84]:
#Task 4 Loading to CSV

In [101]:
def load_to_csv(df, output_path):
    df.to_csv(output_path)

In [86]:
# Task 5 Load to DB

In [87]:
def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

In [88]:
# Task 6: Function to Run queries on Database

In [89]:
def run_query(query_statement, sql_connection):
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

In [94]:
url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
csv_path = "C:/Users/krish/OneDrive/Desktop/exchange_rate.csv"
table_attribs = ["Name", "MC_USD_Billion"]
output_path = "C:/Users/krish/Downloads/Largest_banks_data.csv"
db_name = "Banks.db"
table_name = "Largest_banks"
log_file = "C:/Users/krish/Downloads/code_log.txt"

log_progress("Preliminaries complete. Initiating ETL process")

In [ ]:
# Calling extract() 

In [96]:
df = extract(url, table_attribs)
print(df)

log_progress("Data extraction complete. Initiating Transformation process")

                                      Name  MC_USD_Billion
0                           JPMorgan Chase          432.92
1                          Bank of America          231.52
2  Industrial and Commercial Bank of China          194.56
3               Agricultural Bank of China          160.68
4                                HDFC Bank          157.91
5                              Wells Fargo          155.87
6                                     HSBC          148.90
7                           Morgan Stanley          140.83
8                  China Construction Bank          139.82
9                            Bank of China          136.81


In [97]:
# Calling transform()

In [98]:
df = transform(df, csv_path)
print(df)

log_progress("Data transformation complete. Initiating Loading process")

                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                                     HSBC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2          180.94        16138.75

In [99]:
#Calling load to csv

In [102]:
load_to_csv(df, output_path)

log_progress("Data saved to CSV file")

In [103]:
# SQL connection

In [104]:
sql_connection = sqlite3.connect(db_name)

log_progress("SQL Connection initiated")

In [105]:
# Calling ;load to db

In [106]:
load_to_db(df, sql_connection, table_name)

log_progress("Data loaded to Database as a table, Executing queries")

In [107]:
# Printing entire table

In [111]:
query_statement = f"SELECT * from {table_name}"
run_query(query_statement, sql_connection)

SELECT * from Largest_banks
                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                                     HSBC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2    

In [114]:
query_statement2 = f"SELECT AVG(MC_GBP_Billion) FROM {table_name}"
run_query(query_statement2, sql_connection)

SELECT AVG(MC_GBP_Billion) FROM Largest_banks
   AVG(MC_GBP_Billion)
0              151.987


In [116]:
query_statement3 = f"SELECT NAME FROM {table_name} LIMIT 5"
run_query(query_statement3, sql_connection)

log_progress("Process Complete")

SELECT NAME FROM Largest_banks LIMIT 5
                                      Name
0                           JPMorgan Chase
1                          Bank of America
2  Industrial and Commercial Bank of China
3               Agricultural Bank of China
4                                HDFC Bank


In [117]:
sql_connection.close()

log_progress("Server Connection Closed")

In [118]:
# TASK 7 Verify log entries

In [119]:
with open(log_file, "r") as log:
    LogContent = log.read()
    print(LogContent)

2024-Jul-28-19:21:37 : Preliminaries complete. Initiating ETL process
2024-Jul-28-19:22:44 : Data extraction complete. Initiating Transformation process
2024-Jul-28-19:24:25 : Preliminaries complete. Initiating ETL process
2024-Jul-28-19:26:14 : Preliminaries complete. Initiating ETL process
2024-Jul-28-19:28:33 : Preliminaries complete. Initiating ETL process
2024-Jul-28-19:28:58 : Preliminaries complete. Initiating ETL process
2024-Jul-28-19:29:46 : Preliminaries complete. Initiating ETL process
2024-Jul-28-19:29:50 : Data extraction complete. Initiating Transformation process
2024-Jul-28-19:38:08 : Preliminaries complete. Initiating ETL process
2024-Jul-28-19:38:54 : Data extraction complete. Initiating Transformation process
2024-Jul-28-19:40:03 : Preliminaries complete. Initiating ETL process
2024-Jul-28-19:40:13 : Data extraction complete. Initiating Transformation process
2024-Jul-28-19:40:35 : Preliminaries complete. Initiating ETL process
2024-Jul-28-19:40:36 : Data extraction